# Import libraries

In [1]:
import os
import logging

# Suppress TensorFlow logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 0 = all messages are logged (default), 1 = INFO, 2 = WARNING, 3 = ERROR
logging.getLogger('tensorflow').setLevel(logging.ERROR)
import time  # Import the time module

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import tensorflow as tf

from helpers import parse_variables, get_risk_level, map_to_color, simulate_quant_trait
from models import ols_regression, manhattan_linear, gc
from deep_learning_models import abyss, deep_abyss

from scipy.stats import t
from scipy import stats
from scipy.stats import entropy

from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from tensorflow.keras import Input, Model, layers, regularizers
from tensorflow.keras.layers import Input, Dense


In [3]:
risks = list(naming_dict.keys())

In [4]:
# Load genotype

In [5]:
dict = parse_variables('geno_simulation.txt')
G = int(dict['G'])
L = int(dict['L'])
c = int(dict['c'])
if 'k' not in globals():
    k = int(dict['k'])
    
if 'M' not in globals():
    M = float(dict['M'])
HWE = int(dict['HWE'])

nr_humans = int(dict['nr_humans'])
nr_snps = int(dict['nr_snps'])
bottleneck_nr = int(dict['bottleneck_nr'])

if 'tools' not in globals():
    tools = ['PCA', 'abyss_counted', 'abyss', 'no_corr']


if 'scenarios' not in globals():
    scenarios = ['snp_effect',
                 'linear_continuous',
                 'non_linear_continuous',
                 'discrete_global',
                 'discrete_localized',
                 'mix_linear_continuous',
                 'mix_non_linear_continuous',
                 'mix_discrete_global',
                 'mix_discrete_localized']

if 'very_rare_threshold_L' not in globals():
    very_rare_threshold_L = float(dict['very_rare_threshold_L'])
if 'very_rare_threshold_H' not in globals():
    very_rare_threshold_H = float(dict['very_rare_threshold_H'])
if 'rare_threshold_L' not in globals():
    rare_threshold_L = float(dict['rare_threshold_L'])
if 'rare_threshold_H' not in globals():
    rare_threshold_H = float(dict['rare_threshold_H'])
if 'common_threshold_L' not in globals():
    common_threshold_L = float(dict['common_threshold_L'])
if 'common_threshold_H' not in globals():
    common_threshold_H = float(dict['common_threshold_H'])

number_of_snps = (G*L)/2 # one loci per chromosome
number_of_individuals = c*k*k


very_rare = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/01_veryrare_genotype_AF_{very_rare_threshold_L}_{very_rare_threshold_H}.pkl")
rare = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/01_rare_genotype_AF_{rare_threshold_L}_{rare_threshold_H}.pkl")
common = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/01_common_genotype_AF_{common_threshold_L}_{common_threshold_H}.pkl")

if 'to_analyze' not in globals():
    to_analyze = "complete"

if to_analyze == "complete":
    complete = pd.concat([common, rare, very_rare], axis=1)
elif to_analyze == "common":
    complete = common.copy()
    
elif to_analyze == "rare":
    complete = rare.copy()
elif to_analyze == "very_rare":
    complete = very_rare.copy()
    
complete = ((complete*2)-1)


In [ ]:
# Names of the environmental risks
naming_dict = {
    "no_risk": "no environmental risk",
    "NW_risk": "Smooth linear North-West environmental risk",
    "N_risk" : "Smooth linear North environmental risk",
    "blob_risk": "Localised big blob risk",
    "center_risk": "Localised big central risk",
    "big_square_risk": "big square risk",
    "square_risk" : "Tiny square risk",
    'hi_square_risk' : "Tiny square risk",
    "hi_gauss_blob_risk": "Global Gaussian Risk",
    "two_square_risk": "Two tiny risks",
    "gauss_blob_risk" : "Gaussian Risk",
    "three_square_risk": "Three localized Risks",
    "mid_mid_square_risk": "Mid square risk"
}

# Names of the environmental risks
naming_dict = {
    "no_risk": "no environmental risk",
    "NW_risk": "Smooth linear North-West environmental risk",
    "blob_risk": "Localised big blob risk",
    'hi_square_risk' : "Tiny square risk",
    "three_square_risk": "Three localized Risks",
    "hi_gauss_blob_risk": "Global Gaussian Risk",
}

In [6]:
# Load population

In [7]:
"""
populations = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/phenotype/01_population_structure.pkl")
populations['population_number'] = populations['populations'].str.extract('(\d+)').astype(int)
# Calculating X and Y coordinates
populations['x'] = ((populations['population_number'] - 1) % k) + 1
populations['y'] = ((populations['population_number'] - 1) // k) + 1
populations['z'] = 0.5
populations['population'] = populations['population_number']/(k*k)
palette = [map_to_color(x, y, z, populations) for x, y, z in zip(populations['x'], populations['y'], populations['z'])]

# Standardize the vector
std_dev = np.std(populations['x'])
mean = np.mean(populations['x'])
populations['x'] = np.round((populations['x'] - mean) / std_dev,2)

std_dev = np.std(populations['y'])
mean = np.mean(populations['y'])
populations['y'] = np.round((populations['y'] - mean) / std_dev,2)

# Check the grid
df_agg = populations.groupby(['x', 'y']).agg({'population': 'mean'}).reset_index()

# Now, pivot the aggregated DataFrame
grid_df = df_agg.pivot(index='y', columns='x', values='population')


heatmap = sns.heatmap(grid_df, cmap=palette, linewidths=.5, square=True, cbar=False)

# Add a title to the heatmap
plt.title('Population Grid', fontsize=16)
plt.gca().invert_yaxis()  # Sometimes it's necessary to invert the y-axis for correct orientation
#plt.show()
plt.close()
"""

'\npopulations = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/phenotype/01_population_structure.pkl")\npopulations[\'population_number\'] = populations[\'populations\'].str.extract(\'(\\d+)\').astype(int)\n# Calculating X and Y coordinates\npopulations[\'x\'] = ((populations[\'population_number\'] - 1) % k) + 1\npopulations[\'y\'] = ((populations[\'population_number\'] - 1) // k) + 1\npopulations[\'z\'] = 0.5\npopulations[\'population\'] = populations[\'population_number\']/(k*k)\npalette = [map_to_color(x, y, z, populations) for x, y, z in zip(populations[\'x\'], populations[\'y\'], populations[\'z\'])]\n\n# Standardize the vector\nstd_dev = np.std(populations[\'x\'])\nmean = np.mean(populations[\'x\'])\npopulations[\'x\'] = np.round((populations[\'x\'] - mean) / std_dev,2)\n\nstd_dev = np.std(populations[\'y\'])\nmean = np.mean(populations[\'y\'])\npopulations[\'y\'] = np.round((populations[\'y\'] - mean) / std_dev,2)\n\n# Check the grid\ndf_agg = populations.groupby([\'x\

In [8]:
populations = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/phenotype/01_population_structure.pkl")

In [9]:
true_p2s = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/02_true_p2_via_true_pop.pkl")
true_twopqs = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/02_true_twopq_via_true_pop.pkl")
true_q2s = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/02_true_q2_via_true_pop.pkl")

In [10]:
#onehotpops = pd.get_dummies(populations[['populations']], columns=['populations']).astype(int)

In [11]:
# PCA
nr_complete_PCs = 8
pc_columns = ['PC{}'.format(i) for i in range(1, nr_complete_PCs+1)]

nr_complete_PCs_abyss = 8
pc_columns_abyss = ['PC{}'.format(i) for i in range(1, nr_complete_PCs_abyss+1)]
#PC_common = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/phenotype/PCs/common_genotype_AF_{common_threshold_L}_{common_threshold_H}.pkl")
PC_complete = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/phenotype/PCs/complete_genotypes_AF_0_0.5_.pkl")

In [12]:
pq_file = [f for f in os.listdir(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/") if f.startswith("estimated_2pqs")][0]
p2_file = [f for f in os.listdir(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/") if f.startswith("estimated_p2s")][0]
q2_file = [f for f in os.listdir(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/") if f.startswith("estimated_q2s")][0]

IndexError: list index out of range

In [ ]:
esti_p2 = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/{p2_file}")
esti_twopq = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/{pq_file}")
esti_q2 = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/{q2_file}")

In [ ]:
time_p2 = float(p2_file.split('pop_')[1].split('seconds')[0])
time_q2 = float(q2_file.split('pop_')[1].split('seconds')[0])
time_pq = float(pq_file.split('pop_')[1].split('seconds')[0])
total_abyss_time = np.round(time_p2+time_q2+time_pq,3)
total_abyss_time

In [ ]:
covariate_dictionary_true = {}
for snp in list(complete.columns):
    temp = complete[[snp]].copy()
    temp[f"{snp}_cov_p_q"] = true_p2s[snp] - true_q2s[snp]
    temp[f"{snp}_cov_2pq"] = true_twopqs[snp]
    temp = temp.drop(columns=[snp])
    covariate_dictionary_true[snp] = temp

In [ ]:
covariate_dictionary_esti_0 = {}
for snp in list(complete.columns):
    temp = complete[[snp]].copy()
    temp[f"{snp}_cov_p2"] = esti_p2[snp]
    temp[f"{snp}_cov_q2"] = esti_q2[snp]
    temp[f"{snp}_cov_2pq"] = esti_twopq[snp]
    temp = temp.drop(columns=[snp])
    covariate_dictionary_esti_0[snp] = temp

In [ ]:
covariate_dictionary_combi = {}
for snp in list(complete.columns):
    temp = complete[[snp]].copy()
    temp[f"{snp}_cov_p2"] = esti_p2[snp]
    temp[f"{snp}_cov_q2"] = esti_q2[snp]
    temp[f"{snp}_cov_2pq"] = esti_twopq[snp]
    temp[pc_columns_abyss] = PC_complete[pc_columns_abyss]
    temp = temp.drop(columns=[snp])
    covariate_dictionary_combi[snp] = temp

In [ ]:
mu= np.zeros(complete.shape[0])
beta = np.zeros(complete.shape[1])
noise = 0.9

In [ ]:
def multi(arr, effectsize):
    return [1 * (num + 0.001) * effectsize for num in arr]

def divi(arr, effectsize):
    return [(1 / (num + 0.001)) * effectsize for num in arr]

In [ ]:
effectsize = 1
numbers_af = [float(col.split('_AF_')[1]) for col in common.columns if '_AF_' in col]
snp_names = [col.split('_AF_')[0] for col in common.columns if '_AF_' in col]

beta_values = divi(numbers_af, effectsize)
data = {'snp': snp_names, 'Beta': beta_values}
beta_common = pd.DataFrame(data)
beta_common['maf'] = "common"

numbers_af = [float(col.split('_AF_')[1]) for col in rare.columns if '_AF_' in col]
snp_names = [col.split('_AF_')[0] for col in rare.columns if '_AF_' in col]

beta_values = divi(numbers_af, effectsize)
data = {'snp': snp_names, 'Beta': beta_values}
beta_rare = pd.DataFrame(data)
beta_rare['maf'] = "rare"

numbers_af = [float(col.split('_AF_')[1]) for col in very_rare.columns if '_AF_' in col]
snp_names = [col.split('_AF_')[0] for col in very_rare.columns if '_AF_' in col]

beta_values = divi(numbers_af, effectsize)
data = {'snp': snp_names, 'Beta': beta_values}
beta_very_rare = pd.DataFrame(data)
beta_very_rare['maf'] = "very rare"

betas = pd.concat([beta_common, beta_rare, beta_very_rare], ignore_index=True)

In [ ]:
all_snps = list(complete.columns)
phenos_mono = []
for snp in all_snps:
    index_snp = snp.split('_')[0]
    beta_value = betas.loc[betas['snp'] == index_snp, 'Beta'].values[0]
    phenos_mono.append(complete[snp] * beta_value)

# Converting phenos_mono list of series to DataFrame directly
phenos_mono = pd.concat(phenos_mono, axis=1)
phenos_mono.columns = complete.columns

# Add noise
n = len(phenos_mono)
for snp in list(phenos_mono.columns):
    var_effect = np.var(phenos_mono[snp])
    total_variance = var_effect / 0.001
    var_noise = total_variance - var_effect
    sd_noise = np.sqrt(var_noise)
    # Generate phenotype with noise
    phenos_mono[snp] = phenos_mono[snp] + np.random.normal(0, sd_noise, n)
    std_dev = np.std(phenos_mono[snp])
    mean = np.mean(phenos_mono[snp])
    phenos_mono[snp] = (phenos_mono[snp] - mean) / std_dev


In [ ]:
path_plots = f"data/results/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/plots/"
path_pvals = f"data/results/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/pvals/"

In [ ]:
os.makedirs(path_plots, exist_ok=True)
os.makedirs(path_pvals, exist_ok=True)

In [ ]:
ratio_effects = [0, 0.5, 1]

In [ ]:
noises = [0.1, 0.5]

In [ ]:
iter_risks = risks

In [ ]:
performances = pd.DataFrame()
computing = pd.DataFrame()

for ratio_effect in ratio_effects:
    ratio_environment = 1-ratio_effect
    if ratio_effect == 1:
        iter_risks = ['no_risk']
    for noise in noises:
        for name_risk in iter_risks:
            risk = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/phenotype/environmental_risks/risk_{name_risk}.pkl")
            populations[name_risk] = risk[name_risk]
            df_agg = populations.groupby(['x', 'y']).agg({name_risk: 'mean'}).reset_index()
            grid_df = df_agg.pivot(index='y', columns='x', values=name_risk)
            sns.heatmap(grid_df, cmap='rocket_r', linewidths=.5, square=True, cbar=False)
            
            # Add a title to the heatmap
            plt.title(f"{naming_dict[name_risk]}", fontsize=16)
            plt.gca().invert_yaxis()  # Sometimes it's necessary to invert the y-axis for correct orientation
            plt.savefig(f"{path_plots}/envriskmap_{name_risk}.png", dpi=100)
            #plt.show()
            plt.close()
        
            y = np.array(simulate_quant_trait(mu, np.array(complete), beta, np.array(risk[name_risk]), noise))
            # Calculate the standard deviation and mean
            std_dev = np.std(y)
            mean = np.mean(y)
            # Standardize
            y = (y - mean) / std_dev
            risk['pheno'] = y
            y_snps = phenos_mono.copy()
            for snp in list(phenos_mono.columns):
                y_snps[snp] = phenos_mono[snp]*ratio_effect + y*ratio_environment
            y = y_snps.copy()

            # No correction
            start_time = time.time()
            df_no_corr = manhattan_linear(complete, y)
            end_time = time.time()
            elapsed_time_no_corr = np.round(end_time - start_time,3)

            # Best correction
            start_time = time.time()
            df_best_corr = manhattan_linear(complete, y, risk[[f"{name_risk}"]])

            end_time = time.time()
            elapsed_time_best_corr = np.round(end_time - start_time,3)

            
            # X and Y axis as covaraites
            start_time = time.time()
            pops = populations[['x','y']]
            df_pops = manhattan_linear(complete, y , pops)    
            end_time = time.time()
            elapsed_time_x_y = np.round(end_time - start_time,3)

        
            # p2 - q2 and 2pq as covariates
            start_time = time.time()
            df_p_q_2pq_covs_via_true_pops = manhattan_linear(complete, y, covariate_dictionary_true)
            end_time = time.time()
            elapsed_time_true_ps = np.round(end_time - start_time,3)
    

            # PCs
            start_time = time.time()
            df_PCs = manhattan_linear(complete, y , PC_complete[pc_columns])
            end_time = time.time()
            elapsed_time_pcs = np.round(end_time - start_time,3)

            # Abyss
            start_time = time.time()
            df_abyss_p_q_2pq_covs_via_esti_pop = manhattan_linear(complete, y, covariate_dictionary_esti_0)            
            end_time = time.time()
            elapsed_time_abyss = np.round(end_time - start_time,3) + total_abyss_time
            
            # Combines
            start_time = time.time()
            df_abyss_combined = manhattan_linear(complete, y, covariate_dictionary_combi)
            end_time = time.time()
            elapsed_time_combined = np.round(end_time - start_time,3) + total_abyss_time
            
            #df_pops_onehot = manhattan_linear(complete, y , onehotpops)
            # Create MAF & Pval df
            df_pvals = pd.DataFrame()
            df_pvals[['snp','coefs','AFs']] = df_no_corr[['snp','coefs','AFs']]
            df_pvals['-logPs'] = df_no_corr['-logPs']
            df_pvals["-logP_no_corr"] = df_no_corr['-logPs']
            df_pvals["-logP_best_corr"] = df_best_corr['-logPs']
            
            df_pvals["-logP_true_pop"] = df_pops['-logPs']
            #df_pvals["-logP_true_pop_onehot"] = df_pops_onehot['-logPs']
            df_pvals["-logP_abyss_via_true_pops"] = df_p_q_2pq_covs_via_true_pops['-logPs']
            df_pvals["-logP_abyss_pq2pq_via_esti_pops"] = df_abyss_p_q_2pq_covs_via_esti_pop['-logPs']
            df_pvals["-logP_combined"] = df_abyss_combined['-logPs']
            df_pvals["-logP_PCs"] = df_PCs['-logPs']
            
            # Create QQ plot
            df_bests = pd.DataFrame()
            df_bests["-logP_no_corr"] = np.sort(df_no_corr['-logPs'])
            df_bests["-logP_best_corr"] = np.sort(df_best_corr['-logPs'])
            
            df_bests["-logP_true_pop"] = np.sort(df_pops['-logPs'])
            #df_bests["-logP_true_pop_onehot"] = np.sort(df_pops_onehot['-logPs'])
            df_bests["-logP_abyss_via_true_pops"] = np.sort(df_p_q_2pq_covs_via_true_pops['-logPs'])
            df_bests["-logP_abyss_pq2pq_via_esti_pops"] = np.sort(df_abyss_p_q_2pq_covs_via_esti_pop['-logPs'])
            df_bests["-logP_combined"] = np.sort(df_abyss_combined['-logPs'])
            df_bests["-logP_PCs"] = np.sort(df_PCs['-logPs'])
            
            # Create time df
            df_time = pd.DataFrame()
            df_time["no_corr"] = [elapsed_time_no_corr]
            df_time["best_corr"] = [elapsed_time_best_corr]
            df_time["true_pop"] = [elapsed_time_x_y]
            #df_time["true_pop_onehot"] = np.sort(df_pops_onehot['-logPs'])
            df_time["abyss_via_true_pops"] = [elapsed_time_true_ps]
            df_time["abyss_pq2pq_via_esti_pops"] = [elapsed_time_abyss]
            df_time["combined"] = [elapsed_time_combined] 
            df_time["PCs"] = [elapsed_time_pcs]
        
            # Find the maximum value in the DataFrame excluding inf and NaN
            max_value = df_bests.replace([np.inf, -np.inf], np.nan).max().max()
            
            # Replace inf values with the maximum value found
            df_bests.replace([np.inf, -np.inf], max_value, inplace=True)
            
            # Replace NaN values with the maximum value found
            df_bests.fillna(max_value, inplace=True)
            n = len(df_bests)
            expected_quantiles = np.arange(1, n + 1) / n
            expected_logP = np.sort(-np.log10(expected_quantiles))
            df_bests['expected_logP'] = expected_logP
            df_pvals.to_pickle(f"{path_pvals}/P_vals_risk_{name_risk}_noise_{noise}_ratioeffect_{ratio_effect}.pkl")
            
            
            df_differences = df_bests.subtract(df_bests['-logP_best_corr'], axis=0).abs()
            performances[f"risk_{name_risk}_noise_{noise}_ratioeffect_{ratio_effect}"] = list(df_differences.mean(axis=0))
            computing[f"risk_{name_risk}_noise_{noise}_ratioeffect_{ratio_effect}"] = list(df_time.T[0])
        
            sns.scatterplot(x='expected_logP', y='-logP_no_corr', data=df_bests, color='red', label='no covariate', linewidth=0)
            
            sns.scatterplot(x='expected_logP', y='-logP_true_pop', data=df_bests, color='darkblue', label='True populations x,y axis as covariate', linewidth=0)
            
            #sns.scatterplot(x='expected_logP', y='-logP_true_pop_onehot', data=df_bests, color='lightblue', label='True populations dummies as covariates', linewidth=0)
            
            #sns.scatterplot(x='expected_logP', y='-logP_abyss_via_esti_pops', data=df_bests, color='purple', label='Estimated MAFs as covs', linewidth=0)
            sns.scatterplot(x='expected_logP', y='-logP_abyss_via_true_pops', data=df_bests, color='orange', label='True MAFs as covs', linewidth=0)
            sns.scatterplot(x='expected_logP', y='-logP_abyss_pq2pq_via_esti_pops', data=df_bests, color='yellow', label='Estimates p2, q2, 2pq as covs', linewidth=0)
            sns.scatterplot(x='expected_logP', y='-logP_combined', data=df_bests, color='green', label='combined', linewidth=0)
            
            sns.scatterplot(x='expected_logP', y='-logP_PCs', data=df_bests, color='pink', label=f"{nr_complete_PCs} PCs as covariate", linewidth=0)
            sns.scatterplot(x='expected_logP', y='-logP_best_corr', data=df_bests, color='black', label=f"Best correction", linewidth=0)
            
            # Plot diagonal reference line
            plt.plot([min(df_bests['expected_logP']), max(df_bests['expected_logP'])], 
                     [min(df_bests['expected_logP']), max(df_bests['expected_logP'])], 
                     color='red', linestyle='--')
            
            # Set plot labels and title
            plt.xlabel('Expected')
            plt.ylabel('-Log10(P) Values')
            plt.title(f"QQ Plot of Log Values - {naming_dict[name_risk]}, causal/env = {ratio_effect}, phenotypic noise = {noise}")
            
            # Show legend
            
            plt.legend()
            plt.savefig(f"{path_plots}/qq_risk_{name_risk}_noise_{noise}_ratioeffect_{ratio_effect}.png", dpi=100)
            # Show plot
            #plt.show()
            plt.close()
    iter_risks = risks

os.system(f"rm -rf data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}")
performances['tools'] = list(df_bests.columns)
computing['tools'] = list(df_time.columns)

path_results = f"data/results/"
os.makedirs(path_results, exist_ok=True)
performances.to_pickle(f"{path_results}/{to_analyze}_geno_G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}_total_{total_abyss_time}seconds.pkl")
computing.to_pickle(f"{path_results}/{to_analyze}_geno_G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}_computingtimes.pkl")